In [1]:
library('SelectSim')
library('tidyverse')
library('tictoc')

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] tictoc_1.2        lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1    
 [5] dplyr_1.1.4       purrr_1.0.2       readr_2.1.5       tidyr_1.3.0      
 [9] tibble_3.2.1      ggplot2_3.4.4     tidyverse_2.0.0   SelectSim_0.0.1.3

# Metadata

In [3]:
all_merged_metdata <- readRDS(file='/mnt/ndata/arvind/co_mutation_project/data/processed/all_merged_data_genie_v15.rds')
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples<-readRDS(file='/mnt/ndata/arvind/co_mutation_project/data/processed/genie/msk_one_patient_one_sample_final.rds')

In [4]:
msk_run_gene_list <- readRDS(file='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/data/msk_oncokb_gene_run_list.rds')

In [5]:
head(all_merged_metdata,1)

,sample,class,tumor_run,oncotree,cancer_type,race,cohort,tumor_run_mapped,Tissue,final_tumor_run_mapped
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TCGA-OR-A5J5-01,ACC,ACC,ACC,Adrenocortical Carcinoma,White,TCGA,No mapping,Adrenal Gland,ACC


In [6]:
str(msk_run_gene_list)

List of 6
 $ all   : chr [1:202] "MTOR" "NRAS" "NTRK1" "SDHC" ...
 $ p_505 : chr [1:327] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_468 : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_410 : chr [1:282] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_341 : chr [1:249] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_heme: chr [1:267] "MTOR" "NRAS" "CD58" "NTRK1" ...


## All panel gams runs

In [7]:
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% head(1)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,⋯,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,class,Tumor_run_group,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-MSK-P-0000223,Female,White,Non-Spanish/non-Hispanic,MSK,11142,11142,2016,TRUE,2016,⋯,AASTR,Primary,MSK-IMPACT341,Glioma,Anaplastic Astrocytoma,Primary tumor,AASTR,AASTR,Glioma_low_grade,Glioma


In [8]:
str(msk_run_gene_list)

List of 6
 $ all   : chr [1:202] "MTOR" "NRAS" "NTRK1" "SDHC" ...
 $ p_505 : chr [1:327] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_468 : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_410 : chr [1:282] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_341 : chr [1:249] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_heme: chr [1:267] "MTOR" "NRAS" "CD58" "NTRK1" ...


In [9]:
assay_id<-unique(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples$SEQ_ASSAY_ID)

In [10]:
tumor_groups<-unique(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples$final_run_group)

In [11]:
cleaned_tumor_groups <- tolower(gsub("[[:space:]/-]", "_", tumor_groups))

In [12]:
mapped_tumor_groups <- cleaned_tumor_groups
names(mapped_tumor_groups)<-tumor_groups

In [13]:
str(mapped_tumor_groups)

 Named chr [1:33] "glioma" "melanoma" "salivary_gland_cancer" "leukemia" ...
 - attr(*, "names")= chr [1:33] "Glioma" "Melanoma" "Salivary Gland Cancer" "Leukemia" ...


In [14]:
pan_can_result <- list()

In [16]:
k=1
dir='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/'
tic('Total Time')
for(name in names(msk_run_gene_list)){
    cat('--Running:',name,"--\n")
    if(name=='all'){
        label<-paste(dir,'/results/','msk_panel_',name,'_primary_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='all') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
        label<-paste(dir,'/results/','msk_panel_',name,'_meta_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='all') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
    }
    else if(name=='p_505'){
        label<-paste(dir,'/results/','msk_panel_',name,'_primary_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='p_505') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
        label<-paste(dir,'/results/','msk_panel_',name,'_meta_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='p_505') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
    }
    else if(name=='p_468'){
        label<-paste(dir,'/results/','msk_panel_',name,'_primary_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='p_468') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
        label<-paste(dir,'/results/','msk_panel_',name,'_meta_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='p_468') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
    }
    else if(name=='p_410'){
        label<-paste(dir,'/results/','msk_panel_',name,'_primary_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='p_410') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
        label<-paste(dir,'/results/','msk_panel_',name,'_meta_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='p_410') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
    }
    else if(name=='p_341'){
        label<-paste(dir,'/results/','msk_panel_',name,'_primary_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='p_341') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
        label<-paste(dir,'/results/','msk_panel_',name,'_meta_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='p_341') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
    }
    else{
        name<-'heme'
        label<-paste(dir,'/results/','msk_panel_',name,'_primary_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='heme') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
        label<-paste(dir,'/results/','msk_panel_',name,'_meta_result.rds',sep="")
        msk_result<-readRDS(file = label)
        result_df <- msk_result$result
        result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='heme') %>% mutate('Tumor_run'='Pan-can')
        pan_can_result[[k]]<-result_df
        rm(msk_result)
        k=k+1
    }
 cat('\n---Done---\n')
}
toc()

--Running: all --

---Done---
--Running: p_505 --

---Done---
--Running: p_468 --

---Done---
--Running: p_410 --

---Done---
--Running: p_341 --

---Done---
--Running: p_heme --

---Done---
Total Time: 397.595 sec elapsed


In [17]:
length(pan_can_result)

[1] 12

In [18]:
pan_can_result_msk <- bind_rows(pan_can_result)
rownames(pan_can_result_msk) <- c(1:nrow(pan_can_result_msk))

In [19]:
head(pan_can_result_msk,2)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,nES,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,TP53,PIK3CA,TP53 - PIK3CA,12075,3970,0.4339623,0.14267745,1429,1055.34778,3243,⋯,-296.1027,-13.019157,0,16045,0,ME,TRUE,MSK Primary,all,Pan-can
2,KRAS,EGFR,KRAS - EGFR,5622,1259,0.2020485,0.04524708,36,15.12377,1032,⋯,-192.4016,-6.245916,0,6881,0,ME,TRUE,MSK Primary,all,Pan-can


# Tumor Group Runs

## Primary

In [21]:
tumor_can_result_primary <- list()

In [22]:
k=1
dir='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/'
for(name in names(msk_run_gene_list)){
    cat('--Running:',name,"--\n")
    if(name=='all'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_primary_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='all') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_primary[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }    

    }
    else if(name=='p_505'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' & SEQ_ASSAY_ID=='MSK-IMPACT505'& final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_primary_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='p_505') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_primary[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }    

    }
    else if(name=='p_468'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' & SEQ_ASSAY_ID=='MSK-IMPACT468'& final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_primary_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='p_468') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_primary[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        } 
    }
    else if(name=='p_410'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' & SEQ_ASSAY_ID=='MSK-IMPACT410'& final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_primary_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='p_410') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_primary[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }   

    }
    else if(name=='p_341'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' & SEQ_ASSAY_ID=='MSK-IMPACT341'& final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_primary_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='p_341') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_primary[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }   
    }
    else{
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE=='Primary' & SEQ_ASSAY_ID %in% c('MSK-IMPACT-HEME-400','MSK-IMPACT-HEME-468') & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],'heme',sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_primary_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Primary') %>% mutate('Panel'='heme') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_primary[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }   
    }
 cat('\n---Done---\n')
}

--Running: all --
---Tumor Group--- Glioma --
---Tumor Group--- Glioma --Done--
---Tumor Group--- Melanoma --
---Tumor Group--- Melanoma --Done--
---Tumor Group--- Salivary Gland Cancer --
---Tumor Group--- Salivary Gland Cancer --Done--
---Tumor Group--- Leukemia --
---Tumor Group--- Leukemia --Done--
---Tumor Group--- Anal Cancer --
---Tumor Group--- Anal Cancer --Done--
---Tumor Group--- Appendiceal Cancer --
---Tumor Group--- Appendiceal Cancer --Done--
---Tumor Group--- Bladder Cancer --
---Tumor Group--- Bladder Cancer --Done--
---Tumor Group--- B-Lymphoblastic Leukemia/Lymphoma --
---Tumor Group--- B-Lymphoblastic Leukemia/Lymphoma --Done--
---Tumor Group--- Breast Cancer --
---Tumor Group--- Breast Cancer --Done--
---Tumor Group--- Ovarian Cancer --
---Tumor Group--- Ovarian Cancer --Done--
---Tumor Group--- Renal Cell Carcinoma --
---Tumor Group--- Renal Cell Carcinoma --Done--
---Tumor Group--- Cervical Cancer --
---Tumor Group--- Cervical Cancer --Done--
---Tumor Group--- Bi

In [23]:
length(tumor_can_result_primary)

[1] 109

In [24]:
tumor_can_result_msk_primary <- bind_rows(tumor_can_result_primary)
rownames(tumor_can_result_msk_primary) <- c(1:nrow(tumor_can_result_msk_primary))

In [25]:
head(tumor_can_result_msk_primary,2)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,nES,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,IDH1,ATRX,IDH1 - ATRX,338,222,0.2282242,0.1498987,178,138.0818,215,⋯,47.22300,2.629192,0,560,0,CO,TRUE,MSK Primary,all,glioma
2,TP53,ATRX,TP53 - ATRX,517,222,0.3490885,0.1498987,186,140.9091,222,⋯,42.66629,2.895769,0,739,0,CO,TRUE,MSK Primary,all,glioma


## Metastasis

In [26]:
tumor_can_result_met <- list()

In [27]:
k=1
dir='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/'
for(name in names(msk_run_gene_list)){
    cat('--Running:',name,"--\n")
    if(name=='all'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_meta_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='all') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_met[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }    

    }
    else if(name=='p_505'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' & SEQ_ASSAY_ID=='MSK-IMPACT505'& final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_meta_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='p_505') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_met[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }    

    }
    else if(name=='p_468'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' & SEQ_ASSAY_ID=='MSK-IMPACT468'& final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_meta_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='p_468') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_met[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        } 
    }
    else if(name=='p_410'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' & SEQ_ASSAY_ID=='MSK-IMPACT410'& final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_meta_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='p_410') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_met[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }   

    }
    else if(name=='p_341'){
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' & SEQ_ASSAY_ID=='MSK-IMPACT341'& final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],name,sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_meta_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='p_341') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_met[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }   
    }
    else{
        for(tumor_group in names(mapped_tumor_groups)){
            samples<-(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
                      filter(SAMPLE_TYPE!='Primary' & SEQ_ASSAY_ID %in% c('MSK-IMPACT-HEME-400','MSK-IMPACT-HEME-468') & final_run_group==tumor_group)%>% group_by(final_class) %>% filter(n()>=30) %>% ungroup())$SAMPLE_ID
            if(length(samples)>=30){
                cat('---Tumor Group---',tumor_group,"--\n")
                    finalname<-paste(mapped_tumor_groups[tumor_group],'heme',sep="_")
                    label<-paste(dir,'/results/','msk_panel_',finalname,'_meta_result.rds',sep="")
                    msk_result<-readRDS(file = label)
                    result_df <- msk_result$result
                    result_df <- result_df %>% mutate('Cohort'='MSK Metastasis') %>% mutate('Panel'='heme') %>% mutate('Tumor_run'=mapped_tumor_groups[tumor_group])
                    tumor_can_result_met[[k]]<-result_df
                    rm(msk_result)
                    k=k+1
                cat('---Tumor Group---',tumor_group,"--Done--\n")        
                
            }
        }   
    }
 cat('\n---Done---\n')
}

--Running: all --
---Tumor Group--- Melanoma --
---Tumor Group--- Melanoma --Done--
---Tumor Group--- Salivary Gland Cancer --
---Tumor Group--- Salivary Gland Cancer --Done--
---Tumor Group--- Anal Cancer --
---Tumor Group--- Anal Cancer --Done--
---Tumor Group--- Appendiceal Cancer --
---Tumor Group--- Appendiceal Cancer --Done--
---Tumor Group--- Bladder Cancer --
---Tumor Group--- Bladder Cancer --Done--
---Tumor Group--- B-Lymphoblastic Leukemia/Lymphoma --
---Tumor Group--- B-Lymphoblastic Leukemia/Lymphoma --Done--
---Tumor Group--- Breast Cancer --
---Tumor Group--- Breast Cancer --Done--
---Tumor Group--- Ovarian Cancer --
---Tumor Group--- Ovarian Cancer --Done--
---Tumor Group--- Renal Cell Carcinoma --
---Tumor Group--- Renal Cell Carcinoma --Done--
---Tumor Group--- Cervical Cancer --
---Tumor Group--- Cervical Cancer --Done--
---Tumor Group--- Billary Tract Cancer --
---Tumor Group--- Billary Tract Cancer --Done--
---Tumor Group--- Mature B-Cell Neoplasms --
---Tumor Grou

In [28]:
length(tumor_can_result_met)

[1] 87

In [29]:
tumor_can_result_msk_met <- bind_rows(tumor_can_result_met)
rownames(tumor_can_result_msk_met) <- c(1:nrow(tumor_can_result_msk_met))

In [30]:
head(tumor_can_result_msk_met,2)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,nES,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,NRAS,BRAF,NRAS - BRAF,323,404,0.2868561,0.3587922,17,9.913468,322,⋯,-39.23042,-2.025085,0,727,0,ME,TRUE,MSK Metastasis,all,melanoma
2,NF1,BRAF,NF1 - BRAF,222,404,0.1971581,0.3587922,31,11.569570,222,⋯,-21.98325,-1.617025,0,626,0,ME,TRUE,MSK Metastasis,all,melanoma


# Merge all result into one dataframe

In [31]:
dim(pan_can_result_msk)

[1] 267963     25

In [32]:
dim(tumor_can_result_msk_primary)

[1] 767015     25

In [33]:
dim(tumor_can_result_msk_met)

[1] 457018     25

In [34]:
all_result_merged_msk <- rbind(pan_can_result_msk,tumor_can_result_msk_primary,tumor_can_result_msk_met)

In [35]:
head(all_result_merged_msk)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,nES,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,TP53,PIK3CA,TP53 - PIK3CA,12075,3970,0.43396226,0.14267745,1429,1055.34778,3243,⋯,-296.1027,-13.019157,0,16045,0,ME,TRUE,MSK Primary,all,Pan-can
2,KRAS,EGFR,KRAS - EGFR,5622,1259,0.20204852,0.04524708,36,15.12377,1032,⋯,-192.4016,-6.245916,0,6881,0,ME,TRUE,MSK Primary,all,Pan-can
3,PTEN,TP53,PTEN - TP53,2469,12075,0.08873315,0.43396226,695,468.87573,1655,⋯,-172.3259,-9.869776,0,14544,0,ME,TRUE,MSK Primary,all,Pan-can
4,ARID1A,TP53,ARID1A - TP53,2866,12075,0.10300090,0.43396226,988,630.94088,2150,⋯,-168.1535,-10.759211,0,14941,0,ME,TRUE,MSK Primary,all,Pan-can
5,KRAS,TP53,KRAS - TP53,5622,12075,0.20204852,0.43396226,2773,2418.48500,5013,⋯,-132.9546,-15.025226,0,17697,0,ME,TRUE,MSK Primary,all,Pan-can
6,TP53,APC,TP53 - APC,12075,3486,0.43396226,0.12528302,2302,1905.36483,3239,⋯,130.2265,12.274751,0,15561,0,CO,TRUE,MSK Primary,all,Pan-can


In [ ]:
saveRDS(all_result_merged_msk,file='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/results/all_merged_result_msk_v15.rds')